In [46]:
import os #TO BE CHANGED
import cv2
from glob import glob
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from albumentations import *


In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
drive_path = '/content/drive/MyDrive/PFA' 

In [49]:
def create_dir(path): 
    """Create a directory"""
    if not os.path.exists(path):
        os.mkdir(path)

In [50]:
def load_data(path, split=0.2):
    dataset1_path = path + '/dataset1'
    
    
    images = sorted(glob(f"{dataset1_path}/Images/*.BMP"))
    masks_te = sorted(glob(f"{dataset1_path}/GT_TE/*.bmp"))
    masks_icm = sorted(glob(f"{dataset1_path}/GT_ICM/*.bmp"))
    masks_zp = sorted(glob(f"{dataset1_path}/GT_ZP/*.bmp"))
    
    
    
    """Split data"""
    split_size = int(len(images) * split)
    print(len(masks_zp), len(masks_icm), len(masks_te), len(images))
   
    train_i, test_i = train_test_split(images, test_size=split_size, random_state=42)
    train_te, test_te = train_test_split(masks_te, test_size=split_size, random_state=42)
    train_zp, test_zp = train_test_split(masks_zp, test_size=split_size, random_state=42)
    train_icm, test_icm = train_test_split(masks_icm, test_size=split_size, random_state=42)
   
    return (train_i, test_i), (train_zp, test_zp), (train_icm, test_icm), (train_te, test_te)

In [51]:

def read_this(image_file):
   
    image_src = cv2.imread(image_file)

    image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2GRAY)
    
    return image_src

In [52]:
def enhance_contrast(image_matrix, bins=256):
    """https://msameeruddin.hashnode.dev/image-equalization-contrast-enhancing-in-python"""
    image_flattened = image_matrix.flatten()
    image_hist = np.zeros(bins)

    # frequency count of each pixel
    for pix in image_matrix:
        image_hist[pix] += 1

    # cummulative sum
    cum_sum = np.cumsum(image_hist)
    norm = (cum_sum - cum_sum.min()) * 255
    # normalization of the pixel values
    n_ = cum_sum.max() - cum_sum.min()
    uniform_norm = norm / n_
    uniform_norm = uniform_norm.astype('int')

    # flat histogram
    image_eq = uniform_norm[image_flattened]
    # reshaping the flattened matrix to its original shape
    image_eq = np.reshape(a=image_eq, newshape=image_matrix.shape)

    return image_eq

In [53]:
def findContour(img):
    #finding outline of Blastocoel zone
    '''[inspired by]https://answers.opencv.org/question/223216/python-eliminate-small-contours/'''
    image_contours = np.zeros((img.shape[1], img.shape[0], 1), np.uint8)
    image_binary = np.zeros((img.shape[1], img.shape[0], 1), np.uint8)
    
    contours = cv2.findContours(img, 1, 1)[0]   
    cv2.drawContours(image_contours, contours, -1, (255,255,255))
    
    contours= cv2.findContours(image_contours, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
    cv2.drawContours(image_binary, [max(contours, key = cv2.contourArea)],-1, (255, 255, 255))
    
    #now image_binary has the outline of GT_Blastocoel
    
    return image_binary

In [54]:
def GT_Blastocoel(maskA,maskB,maskC,mergedmasks):
    #finding mask of the Blastocoel zone
    #te,zp,icm

    merge1=cv2.add(maskA,maskB)
    part_merged=cv2.add(merge1,maskC)
    GT_C=mergedmasks+part_merged
    GT_C[GT_C<255]=0

    return GT_C

In [55]:
def addmasks(maskA, maskB, maskC):
    # adding zp,icm,te to deduce full cell mask

    merge1 = cv2.add(maskA, maskB)
    merged = cv2.add(merge1, maskC)
    merged = 255 - merged
    '''https://learnopencv.com/filling-holes-in-an-image-using-opencv-python-c/'''

    th, im_th = cv2.threshold(merged, 220, 255, cv2.THRESH_BINARY_INV);
    im_floodfill = im_th.copy()
    h, w = im_th.shape[:2]
    mask = np.zeros((h + 2, w + 2), np.uint8)
    cv2.floodFill(im_floodfill, mask, (0, 0), 255);
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)
    result = im_th | im_floodfill_inv
    merged = result
    return merged

In [56]:
def generate_mask(mask1, mask2, mask3):
  mask1[mask1 == 255] = 32
  mask2[mask2 == 255] = 64
  mask3[mask3 == 255] = 128
  merge1 = cv2.add(mask1, mask2)
  merged = cv2.add(merge1, mask3)
  merged = 255 - merged
  return merged


In [57]:
def resize_enhance(i, zp, icm, te,f,l):
    W = 256
    H = 256

    i = cv2.resize(i, (W, H))
    zp = cv2.resize(zp, (W, H))
    icm = cv2.resize(icm, (W, H))
    te = cv2.resize(te, (W, H))
    f = cv2.resize(f, (W, H))
    l = cv2.resize(l, (W, H))
    i = enhance_contrast(i)
    i = i.astype('uint8')

    return i, zp, icm, te, f, l

In [58]:
import csv
from google.colab.patches import cv2_imshow

In [59]:
def augment_data(images, masks_zp, masks_icm, masks_te, new_path, train):
    """Performing data augmentation"""
   
    print(len(images))
    for idx, (i, zp, icm, te) in tqdm(enumerate(zip(images, masks_zp, masks_icm, masks_te)), total=len(images)):

        name = i.split("/")[-1].split(".")[0]
        b_name = f"{name} blastocoel.bmp"
    
        nl_path = os.path.join(new_path, "GT_blastocoel/", b_name)

        i = read_this(i)
        zp = read_this(zp)
        icm = read_this(icm)
        te = read_this(te)
        label_mask = generate_mask(zp, icm, te)
        width, height = i.shape[0], i.shape[1]
        # MERGING MASKS = full cell mask

        mergedmask = addmasks(zp, icm, te)
        blastocel = GT_Blastocoel(te,zp,icm,mergedmask)
        cv2.imwrite(nl_path, blastocel)


In [60]:

#Load the dataset
(train_i, test_i), (train_zp, test_zp), (train_icm, test_icm), (train_te, test_te) = load_data(drive_path)

#CHANGE

test_path = drive_path+"/new_data/test/"
train_path= drive_path+"/new_data/train/"
create_dir(train_path)
create_dir(test_path)
create_dir(test_path+"images/")
create_dir(train_path+"images/")
create_dir(test_path+"GT_ZP/")
create_dir(train_path+"GT_ZP/")
create_dir(test_path+"GT_ICM/")
create_dir(train_path+"GT_ICM/")
create_dir(test_path+"GT_TE/")
create_dir(train_path+"GT_TE/")
create_dir(test_path+"GT_full/")
create_dir(train_path+"GT_full/")
create_dir(test_path+"ROI/")
create_dir(train_path+"ROI/")
create_dir(test_path+"label/")
create_dir(train_path+"label/")
create_dir(test_path+ "GT_blastocoel/")
create_dir(train_path+ "GT_blastocoel/")

augment_data(test_i, test_zp, test_icm, test_te,test_path, train=False)
augment_data(train_i, train_zp, train_icm, train_te, train_path, train=True)

249 249 249 249
49


100%|██████████| 49/49 [00:02<00:00, 23.89it/s]


200


100%|██████████| 200/200 [00:07<00:00, 26.22it/s]
